# 🧠 Projeto Connect Four com MCTS e ID3

## 1. Objetivos do Trabalho

Implementar agentes inteligentes para jogar Connect Four usando Monte Carlo Tree Search (MCTS) e árvores de decisão (ID3).  
Adicionalmente, treinar e avaliar o algoritmo ID3 sobre o dataset clássico Iris.

> 📌 *Observação: os resultados foram obtidos com uma simulação de 5 partidas. Devido à aleatoriedade do algoritmo MCTS e à natureza heurística da árvore ID3, os resultados podem variar ligeiramente em execuções futuras.*

---

## 2. Implementações

- MCTS com UCB1, simulação, expansão e retropropagação.
- Geração de dataset (estado, jogada) com MCTS.
- Algoritmo ID3 implementado do zero, com cálculo de entropia e ganho de informação.
- Discretização dos atributos do dataset Iris para aplicar ID3.
- Integração do ID3 ao modo de jogo IA vs IA (em script separado).

---

## 3. Avaliação e Resultados

- **Acurácia da árvore ID3 no dataset Iris:** 98.00%
- **Acurácia da árvore ID3 treinada com dados do MCTS:** 93.17%

### 🔁 Simulação de 5 jogos entre MCTS (X) e ID3 (O):
- ID3 **não podada**: `MCTS(X) - 4 vitórias`, `ID3(O) - 1 vitória`, `empates - 0`
- ID3 **podada** (profundidade 4): `MCTS(X) - 5 vitórias`, `ID3(O) - 0 vitórias`, `empates - 0`

---

## 4. Conclusão

O agente MCTS mostrou-se mais robusto e exploratório, enquanto o ID3 foi eficiente e rápido nas previsões.  
A árvore ID3 obteve boa precisão mesmo com dados complexos, o que mostra sua capacidade de aprender padrões de jogo.

Contudo, observou-se que a versão **não podada da árvore ID3** teve desempenho superior no jogo, vencendo uma partida.  
Já a versão **podada (profundidade 4)**, apesar de mais interpretável, **não conseguiu vencer o MCTS**.


# 🔍 Análise do Projeto Connect Four com MCTS e ID3

Este notebook permite:
- Carregar datasets Iris e MCTS
- Treinar e visualizar árvores ID3
- Simular partidas entre MCTS e ID3

In [ ]:
# 🔁 Importações
import pandas as pd
import random
import math
from collections import Counter, defaultdict
import json



In [2]:
# 📥 Carregar dataset Iris
import pandas as pd  # Ensure pandas is imported
df_iris = pd.read_csv('iris (2).csv')
df_iris.head()

,ID,sepallength,sepalwidth,petallength,petalwidth,class
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [3]:
# 📊 Discretizar atributos contínuos
for col in ['sepallength', 'sepalwidth', 'petallength', 'petalwidth']:
    df_iris[col] = pd.qcut(df_iris[col], q=3, labels=['baixo', 'medio', 'alto'])
df_iris.drop(columns=['ID'], inplace=True)
df_iris.head()

,sepallength,sepalwidth,petallength,petalwidth,class
0,baixo,alto,baixo,baixo,Iris-setosa
1,baixo,medio,baixo,baixo,Iris-setosa
2,baixo,medio,baixo,baixo,Iris-setosa
3,baixo,medio,baixo,baixo,Iris-setosa
4,baixo,alto,baixo,baixo,Iris-setosa


In [23]:
# 🌳 Treinar árvore ID3 com Iris
# Counter and defaultdict are already imported in CELL INDEX: 2

from collections import Counter, defaultdict
iris_records = df_iris.to_dict(orient='records')
attrs = [col for col in df_iris.columns if col != 'class']

def entropy(examples, target_attr):
    c = Counter(x[target_attr] for x in examples)
    total = len(examples)
    return -sum((v / total) * math.log2(v / total) for v in c.values())

def info_gain(examples, attr, target):
    total_entropy = entropy(examples, target)
    subsets = defaultdict(list)
    for ex in examples:
        subsets[ex[attr]].append(ex)
    return total_entropy - sum(
        len(s) / len(examples) * entropy(s, target)
        for s in subsets.values()
    )

def id3(examples, attributes, target):
    labels = [ex[target] for ex in examples]
    if len(set(labels)) == 1:
        return labels[0]
    if not attributes:
        return Counter(labels).most_common(1)[0][0]
    gains = [(a, info_gain(examples, a, target)) for a in attributes]
    best = max(gains, key=lambda x: x[1])[0]
    tree = {best: {}}
    for val in set(ex[best] for ex in examples):
        subset = [ex for ex in examples if ex[best] == val]
        tree[best][val] = id3(subset, [a for a in attributes if a != best], target)
    return tree

iris_tree = id3(iris_records, attrs, 'class')


 

In [24]:
# ✅ Avaliar acurácia
def predict(tree, ex):
    while isinstance(tree, dict):
        attr = next(iter(tree))
        if ex[attr] not in tree[attr]: return None
        tree = tree[attr][ex[attr]]
    return tree

if 'iris_tree' not in globals():
    raise NameError("The variable 'iris_tree' is not defined. Please ensure CELL INDEX: 5 is executed before running this cell.")

correct = sum(predict(iris_tree, r) == r['class'] for r in iris_records)
correct / len(iris_records)

0.98

In [25]:
# ♟️ Carregar e expandir MCTS dataset
mcts_df = pd.read_csv('mcts.dataset.csv.csv')
def expand(row):
    return {f'cell_{i}': row['state'][i] for i in range(42)} | {'move': row['move']}
mcts_records = [expand(r) for _, r in mcts_df.iterrows()]
mcts_attributes = [f'cell_{i}' for i in range(42)]

# Ensure defaultdict is accessible
if 'defaultdict' not in globals():
    from collections import defaultdict

mcts_tree = id3(mcts_records, mcts_attributes, 'move')
mcts_tree

{'cell_17': {'X': {'cell_4': {'X': {'cell_6': {'X': {'cell_1': {'X': {'cell_0': {'X': 2,
          '_': 0}},
        '_': {'cell_5': {'O': 1, '_': {'cell_8': {'O': 5, '_': 1}}}}}},
      'O': {'cell_5': {'O': {'cell_0': {'O': 3, '_': 2}}, '_': 5}},
      '_': {'cell_14': {'X': {'cell_1': {'X': {'cell_16': {'O': {'cell_3': {'O': 6,
              '_': {'cell_5': {'X': 3, '_': 5}}}},
            '_': {'cell_0': {'O': 2, '_': 0}}}},
          'O': {'cell_33': {'X': 6, '_': 5}},
          '_': {'cell_8': {'X': {'cell_7': {'O': 1, '_': 0}},
            'O': 1,
            '_': {'cell_19': {'O': 6, '_': 5}}}}}},
        'O': {'cell_3': {'X': {'cell_0': {'O': {'cell_30': {'O': 6, '_': 2}},
            '_': {'cell_20': {'X': 0,
              'O': 6,
              '_': {'cell_7': {'O': 6, '_': 0}}}}}},
          '_': {'cell_41': {'X': {'cell_1': {'O': 3, '_': 1}},
            '_': {'cell_10': {'O': 6, '_': 3}}}}}},
        '_': {'cell_10': {'X': {'cell_28': {'O': 1,
            '_': {'cell_35': 

In [26]:
# ✅ Avaliar árvore MCTS
correct = sum(predict(mcts_tree, r) == r['move'] for r in mcts_records)
correct / len(mcts_records)

0.9316692667706709

⚠️ Para simulações de jogo, usa `connect_four_ai.py` no terminal.